This is sample web crawling code. In this notebook, the links to articles published by The Print on all the six selected topics are used to get the article text. These articles are added to a dataframe and saved.

In [ ]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import pandas as pd
import re
from selenium import webdriver
from time import sleep
import time
import random
import pickle
import sys

In [ ]:
#Header object
hdr = {'User-Agent': 'Mozilla/5.0'}

In [ ]:
#Defining a function that takes a link and returns the soup version of the html page
def link_to_soup(link_pg):
    req = Request(link_pg, headers=hdr)
    page = urlopen(req) 
    soup_pg = BeautifulSoup(page)
    return soup_pg

In [ ]:
#Defining a function for regex pattern matching
def pattern_finder(pattern, text):
    matches = re.findall(pattern, text, re.IGNORECASE)
    
    if not matches:
        return ["no match"]
    else:
        return matches

In [ ]:
#Function to find the tags in an article
def tags_finder(tags_html):
    pattern = r">([\w ]+)<"
    tags_matches = pattern_finder(pattern, str(tags_html))
    return tags_matches

In [ ]:
#To check if our topic is one of the tags in the article; also extracting the article's title, tag, and description
def topic_article_checker(soup_page, link, topic):    
    
    #Finding the header in the article
    header_obj = soup_page.find("header", class_ = "td-post-title")
    
    #Getting the article's title
    title_html = header_obj.find("h1", class_ = "entry-title")
    pattern = r">([^<>]+)<"
    title = pattern_finder(pattern, str(title_html))[0]
    
    
    #Getting the article's description
    desc_html = header_obj.find("h2", class_ = "td-post-sub-title")
    pattern = r">([^<>]+)<"
    desc = pattern_finder(pattern, str(desc_html))[0]
    
    
    #The date can also be captured here
    date_html = header_obj.find("span", class_ = "update_date")
    pattern = r">([^<>]+) IST<"
    date = pattern_finder(pattern, str(date_html))[0]
        
    
    #Creating a dictionary to map topics and keywords
    topic_keyword_dict = {"farm laws": ['farm', 'agri', 'crop', 'msp', 'paddy'], 
                          "rafale": ['rafale', 'dassault', 'french', 'france'],
                          "article 370": ['article 370', 'srinagar', 'kashmir', 'j&k', 'j&amp;', 'mufti', 'abdullah'],
                          "sabarimala": ['sabarimala', 'ayyappa'],
                          "section 377": ['section 377', 'lgbt', 'gay'],
                          "caa": ["caa", "nrc", "citizen", "shaheen"]
                         }
    
    
    #We can use the article's tags from the website, if the article has tags
    try:
        #Getting the tags from the article
        tags_container = soup_page.find("div", class_ = "td-post-source-tags")
        tags_html = tags_container.find_all("li")
        pattern = r">([^<>]+)<"
        tags = pattern_finder(pattern, str(tags_html))
    
    except:
        #This is if the article doesn't have tags
        
        #Checking if the article's tag or title is listed in the dictionary
        if topic in topic_keyword_dict.keys():
            
            #Fetching the list of keywords for the current topic
            keywords_list = topic_keyword_dict.get(topic)
            
            #Checking if the article is on the current topic
            if (any(keyw in title.lower() for keyw in keywords_list) or 
                any(keyw in link.lower() for keyw in keywords_list)):
            
                    return True, title, desc, date
            
            else:
                    return False, title, desc, date
            
        else:
            raise Exception("Invalid topic")
            
    
    #If the tags were available, then we can use them check to if the article is on the correct topic
    for tag in tags:
        keywords_list = topic_keyword_dict.get(topic)
        
        if any(keyw in tag.lower() for keyw in keywords_list):
            return True, title, desc, date
            
        
    #If none of the conditions have been met, then we can say that this article is not on the current topic
    return False, title, desc, date


In [ ]:
#Defining a function to remove leading and trailing white spaces
def remove_whitespace_trail_lead(text):
    
    #Removing trailing white spaces (retaining line breaks)
    text = re.sub(r"[ \t]+$", "", text)
    
    #Removing leading whitespaces
    text = re.sub(r"^[ \t]+", "", text)
    
    return text

In [ ]:
#Defining a function to take an article's soup page and add the article to the dataframe
def article_adder(soup_page, link, topic):   
    
    #Checking if the article is already in the dataframe
    if not theprint_df['Link'].str.contains(link).any():
        
        #Checking if the article has the relevant topic tag
        check, title, desc, date = topic_article_checker(soup_page, link, topic)
        
        if check:
            
            #Deleting the "contribution" box
            div_obj = soup_page.find("div", class_ = "post_contribute")
            div_obj.clear()
            
            #Finding the author of the article
            author_html = soup_page.find("a", rel = "author")
            pattern = r">([^<>]+)<"
            authors = pattern_finder(pattern, str(author_html))[0]
                        
            
            #Extracting the article text
            #Note that this is not the full name of the class, but BeautifulSoup doesn't require the full class name
            article_body = soup_page.find("div", class_ = "td-post-content")
            
            #Finding the paragraphs within the article body
            article_paras = article_body.find_all(["p", "h3"], class_ = None, recursive=True)
            
            
            #Removing the paras that are links to other articles
            #The find() function returns -1 if a substring is not found in the parent string
            article_text_paras = [i for i in article_paras if (str(i).lower().find("also read")==-1) and 
                                 (str(i).lower().find("read |")==-1) and (str(i).lower().find("read|")==-1) and 
                                 (str(i).lower().find("read :")==-1) and (str(i).lower().find("read:")==-1) and
                                 (str(i).lower().find("watch |")==-1) and (str(i).lower().find("watch|")==-1) and
                                 (str(i).lower().find("watch :")==-1) and (str(i).lower().find("watch:")==-1) and
                                 (str(i).lower().find("see |")==-1) and (str(i).lower().find("see|")==-1) and 
                                 (str(i).lower().find("also see:")==-1) and (str(i).lower().find("also see :")==-1)]
            
            
            #Removing the paras that contain videos embedded
            article_text_paras = [i for i in article_text_paras if str(i).find("<iframe")==-1]
            
            #Extracting the text from these paras
            pattern = r">([^<>]+)<"
            text_paras = pattern_finder(pattern, str(article_text_paras))
                
    
            #Removing leading and trailing whitespaces from the text
            text_paras_cleaned = [remove_whitespace_trail_lead(t) for t in text_paras if t != ', ']
            desc_cleaned = [remove_whitespace_trail_lead(desc)]
            

            #Checking for city/location name in the first paragraph
            pattern = "^[A-Z]{1,15}[A-Za-z\s ()\\\/]{0,15}:([^<>]*$)"
            result = pattern_finder(pattern, text_paras_cleaned[0])
            
            
            #Checking if the city/location name is in the second paragraph
            if result[0] == "no match":
                                
                #Inserting the result in place of the second paragraph if there is a match in the second paragraph
                result = pattern_finder(pattern, text_paras_cleaned[1])
                if result[0] != "no match":
                    text_paras_cleaned[1] = result[0]
            
            #Inserting the result in place of the first paragraph if there is a match
            else:
                text_paras_cleaned[0] = result[0]
            
            
            #Merging the string items in the list of text paras
            text_paras_joined = ' '.join(text_paras_cleaned)
            
            
            #Making sure that there is a description in the article
            if (desc_cleaned[0] != "no match"):                
                text_paras_fin = desc_cleaned + [text_paras_joined]
            
            #In this branch, there's no description
            else:
                text_paras_fin = [text_paras_joined]
                
            
            #Adding the article description to the list
            article_text = ''.join(text_paras_fin)            #Joining without a space here
                               
            
            #Setting the index value where the new row is to be inserted
            if theprint_df.empty:
                row_id = 0
            else:
                row_id = theprint_df.index[-1]
                row_id += 1


            #Adding this article's details to the dataframe
            theprint_df.loc[row_id] = [title] + [link] + [date] + [authors] + [topic] + [article_text]
            
            return 1
            
        else:
            print("Article is not about current topic")
            print("Link to article: ", link)
            return 0
        
    else:
        print("Link is already present in the dataframe")
        print("Link to article: ", link)
        return -1


Let's load the articles from disk.

In [ ]:
def links_unpickle(topic):
    
    #Mapping topics to file names
    
    topic_filename = {"farm laws": 'links to articles_farm_laws.txt', 
                      "rafale": 'links to articles_rafale.txt',
                      "article 370": 'links to articles_article_370.txt',
                      "sabarimala": 'links to articles_sabarimala.txt',
                      "section 377": 'links to articles_section_377.txt',
                      "caa": 'links to articles_caa.txt'
                     }
    
    #Unpickling the relevant file
    with open(topic_filename.get(topic), "rb") as fp:
        links_list = pickle.load(fp)
    
    print("Number of links on the {} topic is {}.".format(topic, len(links_list)))
    
    return links_list
    

We can extract articles from these links. Let's define a function for this.

In [ ]:
def link_to_article(links_to_add, topic):
    
    #Initializing a few variables and lists
    article_counter = 0
    already_present = 0
    other_topics = []
    exceptions_list = []

    #Looping through the links and adding them to the dataframe
    for link in links_to_add:

        try:
            soup_page = link_to_soup(link)

            #Adding the article and collecting the returned value
            check = article_adder(soup_page, link, topic)

            #Tracking the links that were not about our current topic
            if check==0:
                other_topics.append(link)
            elif check==1:
                article_counter += check
            else:
                already_present += 1


                
            sleep(random.randint(3, 15))

            print("Articles added: ", article_counter)


        except Exception as e:    

            #Adding the link to the exceptions list
            exceptions_list.append(link)

            print("Ran into a problem")
            # get the exception information
            error_type, error_obj, error_info = sys.exc_info()      

            #print error info and line that threw the exception                          
            print(error_type, 'Line:', error_info.tb_lineno)
            print("Error object: ", error_obj)

            continue
    
    print("Number of articles added: ", article_counter)
    print("Number of articles that were already in the dataframe: ", already_present)
    
    return other_topics, exceptions_list

In [ ]:
#Creating an empty dataframe to store articles
theprint_df = pd.DataFrame(columns=['Title', 'Link', 'Date', 'Authors', 'Topic', 'Article'])
print(theprint_df.shape)
theprint_df.head()

(0, 6)


,Title,Link,Date,Authors,Topic,Article


Let's unpickle the links topic-wise and add the articles to our dataframe.

#### 1. Farm Laws

In [ ]:
#Setting the topic
topic = 'farm laws'

#Unpickling the links
links_list = links_unpickle(topic)

Number of links on the farm laws topic is 1480.


In [ ]:
#Removing duplicate links
links_list = list(set(links_list))

#Dropping links that have the term 'video' in them
links_list = [l for l in links_list if 'video' not in l]
print("Number of links after cleaning: ", len(links_list))

Number of links after cleaning:  1468


Adding these articles to our dataframe.

In [ ]:
#Collecting the links that were not added to the dataframe
topic_other_list_1, exceptions_links_list_1 = link_to_article(links_list[:15], topic)

Articles added:  1
Articles added:  2
Article is not about current topic
Link to article:  https://theprint.in/india/ayodhya-to-covid-india-is-marching-forward-says-pm-modi-in-open-letter-to-country/432178/
Articles added:  2
Article is not about current topic
Link to article:  https://theprint.in/india/governance/to-return-or-not-kashmiris-driven-out-of-uttarakhand-colleges-dont-know-what-to-do/202143/
Articles added:  2
Article is not about current topic
Link to article:  https://theprint.in/thought-shot/prasoon-joshi-on-being-fair-to-domestic-helpers-valmik-thapar-blames-china-for-ignoring-nature/414726/
Articles added:  2
Article is not about current topic
Link to article:  https://theprint.in/economy/reduce-cost-of-business-reform-land-acquisition-bjp-suggests-plan-for-post-covid-economy/407426/
Articles added:  2
Article is not about current topic
Link to article:  https://theprint.in/india/governance/why-the-cbi-failed-to-prove-its-sohrabuddin-fake-encounter-case/167434/
Article

In [ ]:
theprint_df

,Title,Link,Date,Authors,Topic,Article
0,This Mohali village has no mobile or internet ...,https://theprint.in/health/in-mohali-poor-heal...,"6 June, 2021 1:22 pm",Ananya Bhardwaj,farm laws,The Mohali administration has put up all 341 v...
1,"AAP supports farmers’ Bharat Bandh call too, a...",https://theprint.in/india/aap-supports-farmers...,"6 December, 2020 4:05 pm",ANI,farm laws,Delhi Environment Minister and AAP leader Gopa...
2,Modi govt must treat farmers as businessmen &a...,https://theprint.in/opinion/modi-govt-must-tre...,"13 September, 2019 10:15 am",Shweta Saini,farm laws,If Narendra Modi wants to double farmers’ inco...
3,"Modi’s boat ride got the coverage, but on TV, ...",https://theprint.in/opinion/telescope/modis-bo...,"3 December, 2020 8:26 am",Shailaja Bajpai,farm laws,When a Zee News reporter tried to explain a ga...
4,"Improve farm inputs, equip panchayats to verif...",https://theprint.in/opinion/improve-farm-input...,"7 December, 2020 10:35 am",Srijan Pal Singh,farm laws,In my project to evolve the PURA policy with f...
5,Opposition has finally come together on farm l...,https://theprint.in/opinion/opposition-has-fin...,"28 September, 2020 9:40 am",Zainab Sikander,farm laws,Speaking boldly on Twitter won't work. Opposit...


In [ ]:
print(theprint_df['Link'][5])
print(theprint_df['Article'][5])

https://theprint.in/opinion/opposition-has-finally-come-together-on-farm-legislation-but-that-wont-be-enough/511421/
Speaking boldly on Twitter won't work. Opposition can learn from anti-CAA protests ahead of Bihar and West Bengal assembly elections.E ighteen opposition parties and 31 farmer organisations have come together to oppose Narendra Modi government’s apparently ill-thought out farm laws, which they wanted a select committee to review before passing in the Rajya Sabha due to the evident lacunae in them. But the opposition was bulldozed. Some experts claim that these legislation can eventually lead to formations of monopolies, oligopolies and cartels. But how will the Opposition convince the Indian public about the ill-effects of these farm sector reforms? The opposition parties may have come together – a rare occurrence nowadays – but can they sustain, be effective and mobilise mass uprising? Speaking boldly on Twitter, broadcasting self-recorded videos from central hall and g

In [ ]:
#Collecting the links that were not added to the dataframe
topic_other_list_1, exceptions_links_list_1 = link_to_article(links_list[15:], topic)

Article is not about current topic
Link to article:  https://theprint.in/environment/why-punjabs-farmers-continue-to-burn-stubble-and-poison-delhis-air/319601/
Articles added:  0
Article is not about current topic
Link to article:  https://theprint.in/talk-point/demonetisation-passion-play-pm/14926/
Articles added:  0
Article is not about current topic
Link to article:  https://theprint.in/health/toxic-air-from-farm-fires-could-make-north-indias-covid-fight-deadlier/535904/
Articles added:  0
Articles added:  1
Articles added:  2
Articles added:  3
Articles added:  4
Article is not about current topic
Link to article:  https://theprint.in/india/protection-under-new-land-orders-same-as-himachal-and-uttarakhand-laws-jk-govt-says/535976/
Articles added:  4
Article is not about current topic
Link to article:  https://theprint.in/politics/bjps-big-bengal-plan-for-october-amit-shah-puja-visit-womens-defence-training-protests/515546/
Articles added:  4
Article is not about current topic
Link 

Articles added:  36
Article is not about current topic
Link to article:  https://theprint.in/india/by-what-date-will-chinese-troops-be-kicked-out-of-india-rahul-gandhi-asks-pm-modi/527548/
Articles added:  36
Article is not about current topic
Link to article:  https://theprint.in/plugged-in/india-takes-stand-for-harassed-wives-and-security-forces-get-free-hand-in-valley/72007/
Articles added:  36
Articles added:  37
Articles added:  38
Article is not about current topic
Link to article:  https://theprint.in/opinion/politically-correct/rss-vajpayee-pracharak-narendra-modi/418509/
Articles added:  38
Article is not about current topic
Link to article:  https://theprint.in/india/tamil-culture-essential-for-indias-future-says-rahul-gandhi-at-jallikattu-event-in-tn/585500/
Articles added:  38
Article is not about current topic
Link to article:  https://theprint.in/politics/if-they-have-trouble-with-bharat-mata-ki-jai-bihar-has-trouble-with-them-modi-slams-rjd/536131/
Articles added:  38
Ar

Articles added:  85
Articles added:  86
Articles added:  87
Article is not about current topic
Link to article:  https://theprint.in/science/amazon-the-lungs-of-the-planet-is-on-fire-here-are-5-things-you-need-to-know/281055/
Articles added:  87
Articles added:  88
Article is not about current topic
Link to article:  https://theprint.in/india/governance/west-bengal-plans-new-law-to-tackle-fake-news-on-social-media/70857/
Articles added:  88
Articles added:  89
Article is not about current topic
Link to article:  https://theprint.in/india/delhi-court-denies-deep-sidhus-custody-to-police-in-red-fort-violence-case-filed-by-asi/642522/
Articles added:  89
Article is not about current topic
Link to article:  https://theprint.in/yourturn/reader-view-time-for-govt-to-fulfill-its-minimum-government-maximum-governance-promise/575158/
Articles added:  89
Article is not about current topic
Link to article:  https://theprint.in/world/coronavirus-should-finally-make-us-act-on-illegal-wildlife-trade

Articles added:  127
Article is not about current topic
Link to article:  https://theprint.in/economy/indias-budget-could-widen-to-3-8-of-gdp/346332/
Articles added:  127
Article is not about current topic
Link to article:  https://theprint.in/india/aap-says-cm-arvind-kejriwals-movement-still-restricted-delhi-police-denies-claim/563448/
Articles added:  127
Articles added:  128
Articles added:  129
Articles added:  130
Articles added:  131
Article is not about current topic
Link to article:  https://theprint.in/opinion/women-left-behind-rajasthan-health-insurance-scheme-has-a-gender-gap-study/636101/
Articles added:  131
Articles added:  132
Article is not about current topic
Link to article:  https://theprint.in/india/3-year-old-sujith-trapped-tamil-nadu-borewell-80-hours-dies-body-pulled-out/312712/
Articles added:  132
Articles added:  133
Article is not about current topic
Link to article:  https://theprint.in/talk-point/talk-point-gujarats-income-growth-stayed-same/15436/
Articles

Articles added:  169
Articles added:  170
Articles added:  171
Articles added:  172
Article is not about current topic
Link to article:  https://theprint.in/politics/bjp-set-for-major-overhaul-many-states-likely-to-get-new-party-chiefs-ahead-of-ls-polls/46502/
Articles added:  172
Article is not about current topic
Link to article:  https://theprint.in/opinion/politricks/silence-of-the-gandhis-vs-vocal-ashok-gehlot-bhupesh-baghel-shows-a-new-trend-in-congress/550512/
Articles added:  172
Article is not about current topic
Link to article:  https://theprint.in/opinion/having-limited-mahatma-gandhi-to-sanitation-modi-is-ignoring-his-central-message/262256/
Articles added:  172
Article is not about current topic
Link to article:  https://theprint.in/politics/how-altaf-bukhari-bjps-b-team-in-jk-is-surviving-without-selling-dreams-about-article-370/708862/
Articles added:  172
Article is not about current topic
Link to article:  https://theprint.in/india/education/why-defence-minister-rajna

...

Articles added:  572
Articles added:  573
Article is not about current topic
Link to article:  https://theprint.in/talk-point/legality-of-demonetisation/15002/
Articles added:  573
Article is not about current topic
Link to article:  https://theprint.in/opinion/terrorise-stubble-burning-problem/14822/
Articles added:  573
Article is not about current topic
Link to article:  https://theprint.in/economy/saudi-oil-shock-will-probably-not-matter-as-much-to-the-world-as-it-could-to-india/292611/
Articles added:  573
Articles added:  574
Articles added:  575
Article is not about current topic
Link to article:  https://theprint.in/politics/fuel-price-hike-jobs-womens-safety-are-the-top-suggestions-for-congress-2019-manifesto/141985/
Articles added:  575
Articles added:  576
Articles added:  577
Article is not about current topic
Link to article:  https://theprint.in/india/dont-vote-for-bjp-rakesh-tikait-tells-mahapanchayats-in-poll-bound-west-bengal/621471/
Articles added:  577
Article is not

Articles added:  608
Article is not about current topic
Link to article:  https://theprint.in/politics/timeline-the-twists-turns-in-the-political-battle-for-karnataka/59794/
Articles added:  608
Articles added:  609
Article is not about current topic
Link to article:  https://theprint.in/india/28-terrorists-killed-in-kashmir-in-april-highest-since-the-scrapping-of-article-370/414228/
Articles added:  609
Articles added:  610
Articles added:  611
Articles added:  612
Articles added:  613
Articles added:  614
Articles added:  615
Articles added:  616
Articles added:  617
Article is not about current topic
Link to article:  https://theprint.in/india/start-free-mass-vaccination-stop-central-vista-project-12-opposition-parties-write-to-modi/657317/
Articles added:  617
Article is not about current topic
Link to article:  https://theprint.in/india/punjab-govt-selling-covid-vaccines-to-private-hospitals-for-profit-alleges-sukhbir-badal/671622/
Articles added:  617
Articles added:  618
Article

Articles added:  666
Articles added:  667
Articles added:  668
Articles added:  669
Article is not about current topic
Link to article:  https://theprint.in/india/they-punctured-her-scooter-waited-how-4-men-executed-horrific-hyderabad-rape-murder/329459/
Articles added:  669
Articles added:  670
Article is not about current topic
Link to article:  https://theprint.in/opinion/death-penalty-for-rape-isnt-justice-it-is-vengeance/53519/
Articles added:  670
Article is not about current topic
Link to article:  https://theprint.in/opinion/modis-budget-lures-foreign-capital-but-has-precious-little-for-indians/358861/
Articles added:  670
Articles added:  671
Article is not about current topic
Link to article:  https://theprint.in/ani-press-releases/bangalore-based-property-consultancy-elegance-enterprises-launches-an-online-marketplace-dedicated-to-farmlands/677869/
Articles added:  671
Article is not about current topic
Link to article:  https://theprint.in/economy/these-are-the-5-key-factor

Articles added:  686
Article is not about current topic
Link to article:  https://theprint.in/theprint-otc/selling-psus-fiscal-deficit-short-term-solution-nobel-laureate-abhijit-banerjee/309377/
Articles added:  686
Article is not about current topic
Link to article:  https://theprint.in/pageturner/excerpt/20-years-later-this-is-why-salman-khan-is-still-hated-by-the-bishnois-of-jodhpur/281349/
Articles added:  686
Article is not about current topic
Link to article:  https://theprint.in/politics/10-reasons-why-indias-economy-is-in-the-doldrums-according-to-p-chidambaram/68627/
Articles added:  686
Article is not about current topic
Link to article:  https://theprint.in/talk-point/talk-point-demonetisation-hasnt-even-attacked-1-per-cent-black-money/14933/
Articles added:  686
Articles added:  687
Articles added:  688
Articles added:  689
Articles added:  690
Article is not about current topic
Link to article:  https://theprint.in/politics/kisan-budget-media-freedom-right-to-healthcare-co

Articles added:  729
Articles added:  730
Article is not about current topic
Link to article:  https://theprint.in/economy/modi-govts-rs-50000-crore-export-stimulus-isnt-a-game-changer/291935/
Articles added:  730
Article is not about current topic
Link to article:  https://theprint.in/talk-point/purulia-killings-rss-bjp-workers-under-threat-in-mamatas-west-bengal-or-is-it-orchestrated-panic/66000/
Articles added:  730
Articles added:  731
Article is not about current topic
Link to article:  https://theprint.in/plugged-in/mallya-competes-with-mamata-for-the-headlines/188037/
Articles added:  731
Article is not about current topic
Link to article:  https://theprint.in/thought-shot/ruchir-sharma-sees-economic-democratic-revival-mihir-sharma-on-3-myths-of-eco-downturn/332397/
Articles added:  731
Articles added:  732
Articles added:  733
Articles added:  734
Articles added:  735
Articles added:  736
Articles added:  737
Articles added:  738
Articles added:  739
Number of articles added:  

In [ ]:
#Saving the dataframe to disk
theprint_df.to_pickle("theprint_dataframe", compression="zip")

In [ ]:
#To load the pickled dataframe
theprint_df = pd.read_pickle("theprint_dataframe", compression="zip")
theprint_df.shape

(745, 6)

In [ ]:
print("Number of articles whose topic needs to be checked: ", len(pd.Series(topic_other_list_1).unique()))
print("Number of links that threw exceptions: ", len(exceptions_links_list_1))

Number of articles whose topic needs to be checked:  701
Number of links that threw exceptions:  13


In [ ]:
exceptions_links_list_1

['https://theprint.in/plugged-in/primetime/rajdeep-questions-yechury-over-oppn-tactics-mirror-now-examines-andhras-mystery-disease/563965/',
 'https://theprint.in/plugged-in/primetime/newsx-aaj-tak-on-sc-intervention-in-farmers-govt-impasse-mirror-nows-whatsapp-warning/583769/',
 'https://theprint.in/plugged-in/primetime/aaj-tak-questions-yogis-love-jihad-law-deepak-chaurasia-attacks-pakistan-on-news-nation/568408/',
 'https://theprint.in/plugged-in/primetime/arnab-says-common-people-rejected-bharat-bandh-navika-says-oppn-losing-grip-on-reality/563276/',
 'https://theprint.in/plugged-in/primetime/newsx-zee-news-on-26-11-attacks-ravish-kumar-on-protesting-farmers-left-out-in-the-cold/552817/',
 'https://theprint.in/plugged-in/primetime/singhu-border-shouldnt-become-shaheen-bagh-says-zee-news-ravish-on-khalistanis-at-protests/564645/',
 'https://theprint.in/theprint-profile/china-hand-syl-protest-death-remark-how-haryana-agri-minister-loves-to-target-farmers/606069/',
 'https://theprint.

Since most of these are articles on prime time news, they can be ignored.

Let's review a few random articles.

In [ ]:
sample_df = theprint_df.sample(n=10)
sample_df

,Title,Link,Date,Authors,Topic,Article
204,"SC panel will continue work, whether farmers t...",https://theprint.in/india/sc-panel-will-contin...,"21 January, 2021 7:51 pm",Sravasti Dasgupta,farm laws,The panel met the first batch of farmers Thurs...
644,BJP disrespecting Constitution by making laws ...,https://theprint.in/politics/bjp-disrespecting...,"29 December, 2020 8:00 pm",PTI,farm laws,The former J&amp;K chief minister also said th...
20,No formal talks with govt until harassment sto...,https://theprint.in/india/no-formal-talks-with...,"2 February, 2021 4:25 pm",PTI,farm laws,The farmer union alleged that increased barric...
360,Haryana’s Chautala village sent hundreds to fa...,https://theprint.in/india/how-haryanas-chautal...,"31 May, 2021 8:00 am",Ananya Bhardwaj,farm laws,Former Haryana chief minister O.P. Chautala's ...
492,Why alarm bells are ringing in Punjab over Dee...,https://theprint.in/india/why-alarm-bells-are-...,"6 March, 2021 5:43 pm",Chitleen K Sethi,farm laws,The two have emerged from the farmers’ agitati...
46,"Trump’s disappointment with Supreme Court, and...",https://theprint.in/last-laughs/trumps-disappo...,"14 January, 2021 6:32 pm",Yimkumla Longkumer,farm laws,"The best cartoons of the day, chosen by the ed..."
3,"Modi’s boat ride got the coverage, but on TV, ...",https://theprint.in/opinion/telescope/modis-bo...,"3 December, 2020 8:26 am",Shailaja Bajpai,farm laws,When a Zee News reporter tried to explain a ga...
471,"Farmer leaders reject Modi govt’s proposal, st...",https://theprint.in/india/farmer-leaders-in-a-...,"9 December, 2020 5:07 pm",Sravasti Dasgupta,farm laws,The central govt sent a draft proposal with am...
628,Cracks appear among farmer unions after R-Day ...,https://theprint.in/india/cracks-appear-among-...,"27 January, 2021 6:45 pm",Moushumi Das Gupta,farm laws,Bharatiya Kisan Union (Bhanu) and Rashtriya Ki...
679,Punjab’s protesting farmers playing into Modi ...,https://theprint.in/politics/punjabs-protestin...,"11 November, 2020 11:12 am",Sravasti Dasgupta,farm laws,The Modi government has invited protesting far...


In [ ]:
print(theprint_df['Link'][679])
print(theprint_df['Article'][679])

https://theprint.in/politics/punjabs-protesting-farmers-playing-into-modi-govts-hands-says-state-congress-chief/541831/
The Modi government has invited protesting farmers for talks on 13 November in a bid to end the impasse. But Sunil Jakhar says Centre’s intentions are suspect. Protesting farmers are playing into the hands of the Narendra Modi government by not allowing the passage of passenger trains into the state, Punjab Pradesh Congress Committee President Sunil Jakhar told ThePrint Tuesday.   “While Centre may have invited farmers for talks, the farmers are playing into the hands of the Centre by not allowing passage of passenger trains. Because the Centre is not affected by stoppage of trains in any way and neither do they care about Punjab or its farmers,” said the former Gurdaspur MP. His comments come amid the continued suspension of trains to Punjab by the central government and the ongoing protest by the state’s farmers against the  three contentious farm legislations passe

Delete 'last-laughs' links? https://theprint.in/last-laughs/trumps-disappointment-with-supreme-court-and-what-sitharamans-budget-needs/585400/

#### 2. Rafale

In [ ]:
#To load the pickled dataframe
theprint_df = pd.read_pickle("theprint_dataframe", compression="zip")
theprint_df.shape

(745, 6)

In [ ]:
#Setting the topic
topic = 'rafale'

#Unpickling the links
links_list = links_unpickle(topic)

Number of links on the rafale topic is 1172.


In [ ]:
#Removing duplicate links
links_list = list(set(links_list))

#Dropping links that have the term 'video' in them
links_list = [l for l in links_list if 'video' not in l]
print("Number of links after cleaning: ", len(links_list))

Number of links after cleaning:  1147


Adding these articles to the dataframe

In [ ]:
#Collecting the links that were not added to the dataframe
topic_other_list_2, exceptions_links_list_2 = link_to_article(links_list, topic)

Article is not about current topic
Link to article:  https://theprint.in/talk-point/sc-electoral-bonds-order-some-transparency-or-conveniently-postponing-the-controversy/220738/
Articles added:  0
Article is not about current topic
Link to article:  https://theprint.in/politics/reading-between-the-lines-what-3-losing-bjp-cms-have-been-up-to-on-social-media/163722/
Articles added:  0
Articles added:  1
Article is not about current topic
Link to article:  https://theprint.in/india/chidambaram-slams-bjp-for-their-silence-on-siddiquis-death-soaring-inflation/698491/
Articles added:  1
Articles added:  2
Article is not about current topic
Link to article:  https://theprint.in/opinion/brahmastra/aatmanirbhar-in-defence-cant-be-a-mere-slogan-modi-govt-must-handhold-private-players/462575/
Articles added:  2
Articles added:  3
Article is not about current topic
Link to article:  https://theprint.in/india/punjab-cm-amarinder-singh-is-ready-for-truce-with-navjot-sidhu-if-he-apologises-for-his-at

Articles added:  19
Article is not about current topic
Link to article:  https://theprint.in/talk-point/revoking-pakistans-mfn-status-proportionate-reply-or-modi-exploring-diplomatic-steps-first/193282/
Articles added:  19
Article is not about current topic
Link to article:  https://theprint.in/politics/every-deal-not-bofors-rahul-gandhi-should-apologise-for-false-claims-on-rafale-bjp/163503/
Articles added:  19
Articles added:  20
Article is not about current topic
Link to article:  https://theprint.in/plugged-in/republics-pm-interview-arnabs-questions-invite-attacks-on-opposition-by-modi/214147/
Articles added:  20
Article is not about current topic
Link to article:  https://theprint.in/defence/defence-ministry-approves-purchase-of-33-fighter-aircraft-for-iaf-248-indigenous-missiles/453299/
Articles added:  20
Article is not about current topic
Link to article:  https://theprint.in/plugged-in/indian-tv-news-feels-the-josh-of-airstrikes-pak-media-takes-forest-cover/198165/
Articles ad

Articles added:  40
Articles added:  41
Articles added:  42
Articles added:  43
Articles added:  44
Article is not about current topic
Link to article:  https://theprint.in/opinion/heres-why-you-will-watch-arnab-goswami-navika-kumar-long-after-election-results-are-out/310364/
Articles added:  44
Articles added:  45
Article is not about current topic
Link to article:  https://theprint.in/india/citizenship-amendment-act-is-not-anti-muslim-says-rajnath-singh/336706/
Articles added:  45
Articles added:  46
Articles added:  47
Articles added:  48
Article is not about current topic
Link to article:  https://theprint.in/opinion/india-could-emerge-as-the-global-power-the-world-has-been-waiting-for-after-covid/436814/
Articles added:  48
Article is not about current topic
Link to article:  https://theprint.in/opinion/pharma-firms-and-diagnostics-developers-need-to-unite-they-are-catalysts-for-saving-lives/698641/
Articles added:  48
Article is not about current topic
Link to article:  https://t

Articles added:  67
Article is not about current topic
Link to article:  https://theprint.in/opinion/heres-what-narendra-modi-really-wants-to-say-in-mann-ki-baat-but-cant/169408/
Articles added:  67
Article is not about current topic
Link to article:  https://theprint.in/defence/india-to-stick-to-its-guns-firm-on-russian-s-400-deal-despite-us-pressure-to-back-out/247760/
Articles added:  67
Article is not about current topic
Link to article:  https://theprint.in/opinion/brahmastra/india-can-learn-from-azerbaijan-how-to-spend-smart-on-military-when-budget-is-thin/620248/
Articles added:  67
Article is not about current topic
Link to article:  https://theprint.in/politics/amit-shah-siddaramaiah-govt-karnataka-divide-hindus/45056/
Articles added:  67
Article is not about current topic
Link to article:  https://theprint.in/defence/masood-azhars-jaish-e-mohammad-is-a-potent-force-in-kashmir/193051/
Articles added:  67
Article is not about current topic
Link to article:  https://theprint.in/

Article is not about current topic
Link to article:  https://theprint.in/economy/why-sri-lankas-economy-faces-a-painful-reset-in-the-coming-months/437591/
Articles added:  99
Articles added:  100
Article is not about current topic
Link to article:  https://theprint.in/politics/no-one-can-counter-pm-modi-for-next-25-years-shiv-sena/239666/
Articles added:  100
Article is not about current topic
Link to article:  https://theprint.in/india/governance/rs-13-crore-gst-fraud-busted-after-up-cops-sales-tax-depts-joint-raid-in-lucknow-noida/221323/
Articles added:  100
Articles added:  101
Article is not about current topic
Link to article:  https://theprint.in/politics/left-out-of-up-alliance-congress-gives-up-plan-to-include-bsp-in-mp-cabinet/178295/
Articles added:  101
Article is not about current topic
Link to article:  https://theprint.in/last-laughs/celebrating-section-377-verdict-and-madhya-pradesh-congress-love-for-cows/112787/
Articles added:  101
Articles added:  102
Articles added:

...

Articles added:  306
Articles added:  307
Article is not about current topic
Link to article:  https://theprint.in/india/governance/ahead-of-putin-visit-gifted-students-from-russia-india-collaborate-on-something-special/129270/
Articles added:  307
Articles added:  308
Article is not about current topic
Link to article:  https://theprint.in/politics/calling-for-discipline-these-days-is-branded-autocratic-modi/110355/
Articles added:  308
Article is not about current topic
Link to article:  https://theprint.in/defence/mechanical-failure-could-be-behind-mirage-2000-accident-death-of-2-pilots/187026/
Articles added:  308
Articles added:  309
Articles added:  310
Article is not about current topic
Link to article:  https://theprint.in/last-laughs/staying-afloat-with-quota-and-a-revolving-cbi-door/176872/
Articles added:  310
Article is not about current topic
Link to article:  https://theprint.in/india/full-text-of-president-kovind-speech-new-india-wants-uninterrupted-accelerated-growth/25

Articles added:  330
Article is not about current topic
Link to article:  https://theprint.in/world/casting-couch-was-just-a-seat-with-benefits-for-all-harvey-weinsteins-lawyers-argue/356404/
Articles added:  330
Article is not about current topic
Link to article:  https://theprint.in/last-laughs/the-lathi-rule-a-gandhi-mukt-congress-and-jammu-and-kashmirs-version-of-democracy/343011/
Articles added:  330
Articles added:  331
Articles added:  332
Article is not about current topic
Link to article:  https://theprint.in/defence/we-hit-target-dont-count-casualties-iaf-chief-on-pakistan-strike/200958/
Articles added:  332
Article is not about current topic
Link to article:  https://theprint.in/plugged-in/election-commission-unsure-but-amit-shah-bats-for-simultaneous-polls/98103/
Articles added:  332
Article is not about current topic
Link to article:  https://theprint.in/plugged-in/a-group-of-ministers-to-tackle-lynching-and-peaceful-protest-a-fundamental-right/87334/
Articles added:  332


Articles added:  350
Article is not about current topic
Link to article:  https://theprint.in/politics/rahul-gandhi-rahul-gandhi-rahul-gandhi-5-reasons-congress-sank-in-delhi/363149/
Articles added:  350
Article is not about current topic
Link to article:  https://theprint.in/politics/the-modest-gunny-bag-might-turn-out-to-be-punjabs-biggest-election-issue/232358/
Articles added:  350
Article is not about current topic
Link to article:  https://theprint.in/last-laughs/abhijit-banerjee-ironic-nobel-win-ravi-shankar-prasad-trashed-statement/306113/
Articles added:  350
Article is not about current topic
Link to article:  https://theprint.in/opinion/modi-monitor/how-modi-speaks-a-different-language-with-pakistan-than-what-vajpayee-did/193618/
Articles added:  350
Articles added:  351
Article is not about current topic
Link to article:  https://theprint.in/opinion/in-2019-modis-novelty-factor-is-gone-from-the-persona-and-the-oratory/190535/
Articles added:  351
Articles added:  352
Article

Articles added:  369
Article is not about current topic
Link to article:  https://theprint.in/opinion/4-reasons-amit-shah-has-been-so-generous-to-nitish-kumar-in-seat-sharing-in-bihar/141145/
Articles added:  369
Articles added:  370
Article is not about current topic
Link to article:  https://theprint.in/last-laughs/rafale-leaks-from-france-and-indias-evm-carriers/635019/
Articles added:  370
Article is not about current topic
Link to article:  https://theprint.in/opinion/the-factivist/the-mess-in-indias-higher-judiciary-is-sadly-of-its-own-making/229978/
Articles added:  370
Article is not about current topic
Link to article:  https://theprint.in/opinion/like-in-all-bad-marriages-shiv-sena-and-bjp-keep-giving-alliance-a-last-shot/197433/
Articles added:  370
Articles added:  371
Articles added:  372
Articles added:  373
Articles added:  374
Article is not about current topic
Link to article:  https://theprint.in/talk-point/citizenship-law-protests-is-the-oppositions-political-respons

Articles added:  383
Article is not about current topic
Link to article:  https://theprint.in/defence/iaf-looking-at-leasing-midair-refuellers-as-efforts-to-purchase-6-go-on-says-bhadauria/517377/
Articles added:  383
Article is not about current topic
Link to article:  https://theprint.in/world/north-korea-locks-down-city-following-report-on-the-first-suspected-case-of-covid/468404/
Articles added:  383
Article is not about current topic
Link to article:  https://theprint.in/theprint-essential/ex-cjis-iaf-officer-brother-anjan-gogoi-too-has-post-retirement-job-secretary-level-post/383482/
Articles added:  383
Article is not about current topic
Link to article:  https://theprint.in/politics/bsy-diary-forged-says-jaitley-accuses-media-of-playing-along-with-opposition/210665/
Articles added:  383
Articles added:  384
Articles added:  385
Article is not about current topic
Link to article:  https://theprint.in/defence/lockheed-martin-says-it-wont-sell-f-21-jets-to-any-other-country-if-iaf

In [ ]:
#Saving the dataframe to disk
theprint_df.to_pickle("theprint_dataframe", compression="zip")
theprint_df.shape

(1133, 6)

In [ ]:
print("Number of articles whose topic needs to be checked: ", len(pd.Series(topic_other_list_2).unique()))
print("Number of links that threw exceptions: ", len(exceptions_links_list_2))

Number of articles whose topic needs to be checked:  755
Number of links that threw exceptions:  4


#### 3. Article 370

In [ ]:
#Setting the topic
topic = 'article 370'

#Unpickling the links
links_list = links_unpickle(topic)

Number of links on the article 370 topic is 2346.


In [ ]:
#Removing duplicate links
links_list = list(set(links_list))

#Dropping links that have the term 'video' in them
links_list = [l for l in links_list if 'video' not in l]
print("Number of links after cleaning: ", len(links_list))

Number of links after cleaning:  2315


Adding these articles to the dataframe.

In [ ]:
#Collecting the links that were not added to the dataframe
topic_other_list_3, exceptions_links_list_3 = link_to_article(links_list, topic)

Articles added:  1
Articles added:  2
Articles added:  3
Articles added:  4
Articles added:  5
Article is not about current topic
Link to article:  https://theprint.in/politics/why-bjp-ally-nitish-kumar-is-silent-on-ram-mandir-bhoomi-pujan/476027/
Articles added:  5
Article is not about current topic
Link to article:  https://theprint.in/opinion/being-indian/win-some-lose-some-2019-for-bjp-was-modi-made-but-caa-protests-turned-up-a-surprise/342401/
Articles added:  5
Articles added:  6
Article is not about current topic
Link to article:  https://theprint.in/ani-press-releases/newj-amps-up-original-content-launches-weekly-talk-show-featuring-celebrated-indians-in-various-fields/705430/
Articles added:  6
Articles added:  7
Article is not about current topic
Link to article:  https://theprint.in/opinion/solution-to-pakistan-nepal-maps-india-pressing-reset-button-on-regional-blocs/476907/
Articles added:  7
Articles added:  8
Articles added:  9
Articles added:  10
Articles added:  11
Arti

Article is not about current topic
Link to article:  https://theprint.in/opinion/for-wealth-chasers-liberation-theology-is-marxist-for-others-its-justice-and-freedom/699101/
Articles added:  54
Article is not about current topic
Link to article:  https://theprint.in/india/ed-books-sharad-pawar-ajit-pawar-under-pmla-in-maharashtra-cooperative-bank-scam-case/296312/
Articles added:  54
Article is not about current topic
Link to article:  https://theprint.in/india/how-pakistani-lt-col-nisar-ahmed-won-over-indian-peers-after-stalling-their-advance-in-1965/275748/
Articles added:  54
Article is not about current topic
Link to article:  https://theprint.in/trawling-twitter/modi-rahul-smriti-sharad-yadav-grieve-side-by-side-for-one-of-a-kind-sushma-swaraj/273704/
Articles added:  54
Articles added:  55
Articles added:  56
Articles added:  57
Articles added:  58
Article is not about current topic
Link to article:  https://theprint.in/opinion/not-all-muslim-nations-love-modi-this-is-why-iran-ma

Article is not about current topic
Link to article:  https://theprint.in/opinion/letter-from-pakistan/imran-khan-slamming-modi-govts-cab-exposes-his-memory-loss/333920/
Articles added:  113
Articles added:  114
Article is not about current topic
Link to article:  https://theprint.in/india/on-caa-protests-in-up-modi-says-vandals-should-know-rights-and-duties-go-hand-in-hand/340732/
Articles added:  114
Articles added:  115
Article is not about current topic
Link to article:  https://theprint.in/india/nrc-must-for-national-security-implemented-in-bengal-amit-shah/299687/
Articles added:  115
Articles added:  116
Articles added:  117
Articles added:  118
Articles added:  119
Articles added:  120
Articles added:  121
Articles added:  122
Articles added:  123
Articles added:  124
Article is not about current topic
Link to article:  https://theprint.in/politics/goodbye-ghulam-nabi-azad-why-congress-stayed-silent-when-modi-cried-opponents-praised-him/602159/
Articles added:  124
Article is no

Articles added:  152
Articles added:  153
Articles added:  154
Article is not about current topic
Link to article:  https://theprint.in/defence/confident-of-getting-a-good-grant-from-modi-govts-budget-says-army-chief-naravane/355713/
Articles added:  154
Article is not about current topic
Link to article:  https://theprint.in/india/session-with-ips-officer-rema-rajeshwari-truly-engrossing/314314/
Articles added:  154
Article is not about current topic
Link to article:  https://theprint.in/politics/5-reasons-why-jharkhand-assembly-election-results-are-crucial-for-bjp/338696/
Articles added:  154
Article is not about current topic
Link to article:  https://theprint.in/politics/lockdown-on-ram-mandir-bhoomi-pujan-day-but-not-eid-bjp-says-mamata-govt-punishing-hindus/472102/
Articles added:  154
Articles added:  155
Article is not about current topic
Link to article:  https://theprint.in/theprint-profile/up-bjp-president-behind-modi-china-war-remark-was-once-a-congress-singh/532351/
Articl

Articles added:  197
Articles added:  198
Article is not about current topic
Link to article:  https://theprint.in/politics/haryanas-jats-love-modi-voters-state-equations-in-mind/308723/
Articles added:  198
Articles added:  199
Articles added:  200
Link is already present in the dataframe
Link to article:  https://theprint.in/india/article-370-farm-laws-nep-modi-govt-plans-booklet-to-showcase-major-reforms-since-2014/575116/
Articles added:  200
Articles added:  201
Articles added:  202
Article is not about current topic
Link to article:  https://theprint.in/diplomacy/in-a-first-indias-dr-reddys-labs-to-supply-a-generic-drug-to-chinas-public-hospitals/298035/
Articles added:  202
Articles added:  203
Articles added:  204
Article is not about current topic
Link to article:  https://theprint.in/opinion/maharashtra-haryana-send-a-message-against-bjp-arrogance-of-power/310822/
Articles added:  204
Article is not about current topic
Link to article:  https://theprint.in/last-laughs/a-tale-

...

Articles added:  1080
Articles added:  1081
Article is not about current topic
Link to article:  https://theprint.in/trawling-twitter/kejriwals-chalisa-zaira-wasims-kashmir-blues-baba-sehgals-ode-to-shakira/359584/
Articles added:  1081
Articles added:  1082
Articles added:  1083
Articles added:  1084
Articles added:  1085
Articles added:  1086
Articles added:  1087
Article is not about current topic
Link to article:  https://theprint.in/opinion/side-effect-of-shivakumar-and-chidambarams-arrest-congress-gets-its-mojo-back/288054/
Articles added:  1087
Articles added:  1088
Articles added:  1089
Article is not about current topic
Link to article:  https://theprint.in/india/governance/rajnath-raises-issue-of-absentee-mps-in-parliament-as-govt-prepares-to-table-citizenship-bill/329544/
Articles added:  1089
Article is not about current topic
Link to article:  https://theprint.in/opinion/modi-govt-claiming-pok-is-great-for-diplomacy-but-wont-mean-much-militarily/293751/
Articles added:  10

Articles added:  1118
Articles added:  1119
Article is not about current topic
Link to article:  https://theprint.in/ani-press-releases/blamglam-aspires-to-be-the-one-stop-destination-for-entertainment-news/701074/
Articles added:  1119
Articles added:  1120
Article is not about current topic
Link to article:  https://theprint.in/india/amit-shah-seeks-support-for-caa-in-bihar-says-nitish-kumar-will-lead-nda-in-state-polls/350453/
Articles added:  1120
Articles added:  1121
Article is not about current topic
Link to article:  https://theprint.in/opinion/global-print/abki-baar-modi-must-stitch-a-better-friendship-with-biden-harris-us-sarkar/540715/
Articles added:  1121
Articles added:  1122
Articles added:  1123
Articles added:  1124
Articles added:  1125
Article is not about current topic
Link to article:  https://theprint.in/india/governance/gnctd-act-will-paralyse-governance-in-delhi-say-76-former-civil-servants/637377/
Articles added:  1125
Articles added:  1126
Article is not about

Articles added:  1183
Articles added:  1184
Articles added:  1185
Articles added:  1186
Link is already present in the dataframe
Link to article:  https://theprint.in/last-laughs/celestial-bodies-need-to-be-fed-and-delhi-tourists-arrive-to-enjoy-kashmir-freeze/303755/
Articles added:  1186
Articles added:  1187
Article is not about current topic
Link to article:  https://theprint.in/defence/army-jawan-killed-another-injured-as-pakistan-violates-ceasefire-along-loc-in-jks-poonch/514386/
Articles added:  1187
Article is not about current topic
Link to article:  https://theprint.in/india/arun-jaitleys-health-condition-still-critical-president-visits-him-at-aiims/277806/
Articles added:  1187
Articles added:  1188
Articles added:  1189
Articles added:  1190
Article is not about current topic
Link to article:  https://theprint.in/opinion/from-priyanka-chopra-to-doordarshan-patriotism-today-is-a-show-of-love-for-indian-military/277440/
Articles added:  1190
Article is not about current topic

Articles added:  1222
Articles added:  1223
Article is not about current topic
Link to article:  https://theprint.in/politics/those-with-history-of-making-bihar-bimaru-wont-be-allowed-to-return-says-pm-modi/529446/
Articles added:  1223
Articles added:  1224
Articles added:  1225
Article is not about current topic
Link to article:  https://theprint.in/plugged-in/primetime/rajdeep-rubika-question-congress-on-poll-losses-republic-bharat-attacks-gupkar-gang/545848/
Articles added:  1225
Articles added:  1226
Article is not about current topic
Link to article:  https://theprint.in/india/public-debate-needed-on-uniform-civil-code-up-to-govt-to-bring-bill-says-hosabale-of-rss/530689/
Articles added:  1226
Article is not about current topic
Link to article:  https://theprint.in/politics/marauding-bjp-sena-listless-opposition-maharashtra-most-lopsided-polls/308614/
Articles added:  1226
Articles added:  1227
Articles added:  1228
Articles added:  1229
Articles added:  1230
Article is not about

Articles added:  1270
Articles added:  1271
Article is not about current topic
Link to article:  https://theprint.in/ani-press-releases/a-story-of-revenge-and-power-meet-the-angre-family-of-cartel/711692/
Articles added:  1271
Article is not about current topic
Link to article:  https://theprint.in/politics/yogi-nitish-duel-on-caa-is-ndas-fixed-game-for-bihar-says-tejashwis-party/537714/
Articles added:  1271
Articles added:  1272
Articles added:  1273
Article is not about current topic
Link to article:  https://theprint.in/opinion/data-shows-upper-caste-voters-of-both-bjp-and-congress-support-core-hindutva-issues/320208/
Articles added:  1273
Articles added:  1274
Articles added:  1275
Articles added:  1276
Article is not about current topic
Link to article:  https://theprint.in/plugged-in/alert-in-kutch-is-tois-lead-express-ht-go-with-pakistan-missile-test-hindu-flags-nrc/284274/
Articles added:  1276
Articles added:  1277
Articles added:  1278
Article is not about current topic
Link

In [ ]:
#Saving the dataframe to disk
theprint_df.to_pickle("theprint_dataframe", compression="zip")
theprint_df.shape

(2414, 6)

#### 4. Sabarimala

In [ ]:
#Setting the topic
topic = 'sabarimala'

#Unpickling the links
links_list = links_unpickle(topic)

Number of links on the sabarimala topic is 429.


In [ ]:
#Removing duplicate links
links_list = list(set(links_list))

#Dropping links that have the term 'video' in them
links_list = [l for l in links_list if 'video' not in l]
print("Number of links after cleaning: ", len(links_list))

Number of links after cleaning:  418


Adding these articles to the dataframe.

In [ ]:
#Collecting the links that were not added to the dataframe
topic_other_list_4, exceptions_links_list_4 = link_to_article(links_list, topic)

Article is not about current topic
Link to article:  https://theprint.in/politics/how-pm-modi-transforms-into-campaigner-modi-as-he-mixes-official-trips-with-politics/191484/
Articles added:  0
Article is not about current topic
Link to article:  https://theprint.in/last-laughs/questionable-answers-and-keralas-wall-of-unity/172527/
Articles added:  0
Articles added:  1
Article is not about current topic
Link to article:  https://theprint.in/ani-press-releases/chetu-expands-global-offices-on-three-continents-amid-sustained-rapid-growth/697172/
Articles added:  1
Article is not about current topic
Link to article:  https://theprint.in/politics/single-seat-in-bengal-dip-in-kerala-tally-why-muslim-parties-didnt-do-well-this-time/651929/
Articles added:  1
Article is not about current topic
Link to article:  https://theprint.in/ani-press-releases/the-international-institute-of-academic-research-and-publications-in-india-launches-open-access-publishing-hub-with-f1000/702351/
Articles added: 

Articles added:  9
Link is already present in the dataframe
Link to article:  https://theprint.in/judiciary/a-year-counting-clock-ticking-but-sc-verdict-yet-awaited-on-over-20-pleas-on-article-370-scrapping/577946/
Articles added:  9
Articles added:  10
Link is already present in the dataframe
Link to article:  https://theprint.in/judiciary/mehbooba-muftis-daughter-seeks-sc-permission-for-her-mother-to-conduct-political-activity/509010/
Articles added:  10
Articles added:  11
Link is already present in the dataframe
Link to article:  https://theprint.in/india/2019-communication-blackout-pakistan-why-2020-saw-spike-in-kashmir-youth-joining-militants/580019/
Articles added:  11
Link is already present in the dataframe
Link to article:  https://theprint.in/last-laughs/bollywoods-silence-over-tanushrees-allegations-india-celebrates-the-sabarimala-verdict/126883/
Articles added:  11
Link is already present in the dataframe
Link to article:  https://theprint.in/india/doubling-farmers-income-

Articles added:  26
Articles added:  27
Article is not about current topic
Link to article:  https://theprint.in/india/governance/bihar-dgp-gupteshwar-pandeys-case-rare-approval-of-vrs-plea-by-ias-ips-takes-month-or-two/509039/
Articles added:  27
Article is not about current topic
Link to article:  https://theprint.in/opinion/how-nehru-and-yogi-adityanath-differed-on-naming-cities-from-chandigarh-to-prayagraj/151329/
Articles added:  27
Article is not about current topic
Link to article:  https://theprint.in/india/governance/lateral-entry-experts-have-become-like-any-ias-officer-a-year-into-recruitment/505476/
Articles added:  27
Article is not about current topic
Link to article:  https://theprint.in/india/governance/women-challenge-nizamuddin-dargah-ban-which-is-in-place-because-dead-can-see-them-naked/160743/
Articles added:  27
Articles added:  28
Article is not about current topic
Link to article:  https://theprint.in/india/governance/modi-govt-plans-india75-booklet-will-list-pro

Articles added:  38
Article is not about current topic
Link to article:  https://theprint.in/india/church-committed-land-fraud-in-andhra-canadian-donors-allege-fight-to-repair-damage/548563/
Articles added:  38
Article is not about current topic
Link to article:  https://theprint.in/plugged-in/pm-modi-on-front-pages-et-mint-tick-upturn-in-manufacturing-hotel-cons-in-hindu-nie/359328/
Articles added:  38
Article is not about current topic
Link to article:  https://theprint.in/opinion/how-three-key-supreme-court-judgments-shook-the-faith-of-scs-sts-and-obcs-in-one-year/184178/
Articles added:  38
Article is not about current topic
Link to article:  https://theprint.in/india/bjp-mps-say-save-india-urge-modi-govt-to-bring-in-population-control-law/509248/
Articles added:  38
Articles added:  39
Article is not about current topic
Link to article:  https://theprint.in/politics/bjp-provoking-tensions-in-temples-andhra-cm-chandrababu-naidu-at-mega-opposition-rally/180261/
Articles added:  39
A

Articles added:  53
Articles added:  54
Articles added:  55
Articles added:  56
Articles added:  57
Article is not about current topic
Link to article:  https://theprint.in/judiciary/7-11-blasts-14-yrs-on-convicts-still-blame-indian-mujahideen-rely-on-police-documents/578461/
Articles added:  57
Article is not about current topic
Link to article:  https://theprint.in/opinion/if-we-had-filmfare-awards-for-real-life-these-would-be-the-winners-of-2018/170459/
Articles added:  57
Article is not about current topic
Link to article:  https://theprint.in/plugged-in/primetime/pms-tears-move-news-anchors-aaj-tak-tears-into-uddhav-governments-attack-on-desh-prem/602301/
Articles added:  57
Article is not about current topic
Link to article:  https://theprint.in/opinion/jaggi-vasudev-is-the-ducati-riding-guru-for-the-modi-age/206289/
Articles added:  57
Link is already present in the dataframe
Link to article:  https://theprint.in/india/not-just-farm-laws-sugarcane-price-delayed-payments-also-cau

Articles added:  66
Article is not about current topic
Link to article:  https://theprint.in/politics/governance-not-ideology-how-keralas-ldf-managed-a-historic-return-to-power-under-pinarayi/650907/
Articles added:  66
Articles added:  67
Article is not about current topic
Link to article:  https://theprint.in/plugged-in/tharoor-calls-kejriwal-a-rude-name-on-cnn-news-18-arnab-goswamis-gandhi-vadra-expose/349190/
Articles added:  67
Articles added:  68
Article is not about current topic
Link to article:  https://theprint.in/health/this-delhi-doctor-doesnt-wear-full-ppe-when-treating-covid-19-patients-heres-why/505572/
Articles added:  68
Link is already present in the dataframe
Link to article:  https://theprint.in/politics/set-up-a-jpc-to-look-into-rafale-scam-says-aap-petitioner-sanjay-singh/163822/
Articles added:  68
Articles added:  69
Link is already present in the dataframe
Link to article:  https://theprint.in/opinion/if-india-keeps-diluting-its-liberal-character-the-west-will-

Article is not about current topic
Link to article:  https://theprint.in/ani-press-releases/best-crop-science-llp-becomes-first-agrochemical-company-in-india-to-manufacture-trifloxystrobin-technical/701871/
Articles added:  91
Articles added:  92
Link is already present in the dataframe
Link to article:  https://theprint.in/india/kashmiri-pandits-body-chief-on-fast-unto-death-demands-jk-govt-address-communitys-demands/507991/
Articles added:  92
Article is not about current topic
Link to article:  https://theprint.in/talk-point/tipu-jayanti-goes-to-karnataka-hc-sabarimala-style-flashpoint-or-real-historical-debate/147370/
Articles added:  92
Articles added:  93
Article is not about current topic
Link to article:  https://theprint.in/opinion/afghanistan-is-indias-one-reliable-neighbour-but-hekmatyars-surprise-entry-can-change-this/510017/
Articles added:  93
Articles added:  94
Articles added:  95
Article is not about current topic
Link to article:  https://theprint.in/politics/bjp-not-

Articles added:  106
Article is not about current topic
Link to article:  https://theprint.in/politics/congress-promise-of-law-protecting-sabarimala-tradition-not-a-u-turn-says-oommen-chandy/630499/
Articles added:  106
Article is not about current topic
Link to article:  https://theprint.in/opinion/bjp-leaders-cant-be-faulted-for-rushing-to-kumbh-mela-they-would-be-foolish-not-to/187181/
Articles added:  106
Article is not about current topic
Link to article:  https://theprint.in/politics/how-left-in-kerala-is-invoking-both-marx-and-market-to-keep-power-in-this-election/632376/
Articles added:  106
Article is not about current topic
Link to article:  https://theprint.in/national-interest/kanwarias-vs-halloweeners-indias-new-political-faultline/147733/
Articles added:  106
Link is already present in the dataframe
Link to article:  https://theprint.in/opinion/pakistans-upgrade-of-gilgit-baltistan-is-linked-to-the-india-china-stand-off/506599/
Articles added:  106
Article is not about cu

In [ ]:
#Saving the dataframe to disk
theprint_df.to_pickle("theprint_dataframe", compression="zip")
theprint_df.shape

(2524, 6)

#### 5. Section 377

In [ ]:
#Setting the topic
topic = 'section 377'

#Unpickling the links
links_list = links_unpickle(topic)

Number of links on the section 377 topic is 430.


In [ ]:
#Removing duplicate links
links_list = list(set(links_list))

#Dropping links that have the term 'video' in them
links_list = [l for l in links_list if 'video' not in l]
print("Number of links after cleaning: ", len(links_list))

Number of links after cleaning:  398


Adding these articles to the dataframe.

In [ ]:
#Collecting the links that were not added to the dataframe
topic_other_list_5, exceptions_links_list_5 = link_to_article(links_list, topic)

Article is not about current topic
Link to article:  https://theprint.in/politics/how-pm-modi-transforms-into-campaigner-modi-as-he-mixes-official-trips-with-politics/191484/
Articles added:  0
Article is not about current topic
Link to article:  https://theprint.in/last-laughs/questionable-answers-and-keralas-wall-of-unity/172527/
Articles added:  0
Link is already present in the dataframe
Link to article:  https://theprint.in/politics/shashi-tharoors-article-kicks-up-a-new-storm-on-sabarimala-splits-opinion/148163/
Articles added:  0
Article is not about current topic
Link to article:  https://theprint.in/ani-press-releases/chetu-expands-global-offices-on-three-continents-amid-sustained-rapid-growth/697172/
Articles added:  0
Article is not about current topic
Link to article:  https://theprint.in/opinion/i-am-a-sexual-abuse-survivor-no-i-am-not-a-woman/641099/
Articles added:  0
Link is already present in the dataframe
Link to article:  https://theprint.in/india/sabarimala-pilgrimag

Articles added:  6
Articles added:  7
Link is already present in the dataframe
Link to article:  https://theprint.in/features/techies-may-have-answer-to-sabarimala-ayodhya-rows-but-are-temples-or-mosques-ready/178685/
Articles added:  7
Link is already present in the dataframe
Link to article:  https://theprint.in/last-laughs/bollywoods-silence-over-tanushrees-allegations-india-celebrates-the-sabarimala-verdict/126883/
Articles added:  7
Article is not about current topic
Link to article:  https://theprint.in/india/uttarakhand-hc-questions-constitutionality-of-ban-on-slaughterhouses-in-haridwar-district/698007/
Articles added:  7
Articles added:  8
Article is not about current topic
Link to article:  https://theprint.in/opinion/guide-for-nris-resettling-in-india-after-trump-throws-them-out/159397/
Articles added:  8
Article is not about current topic
Link to article:  https://theprint.in/india/long-way-to-go-in-india-say-activists-on-zero-tolerance-to-female-genital-mutilation-day/6001

Articles added:  17
Article is not about current topic
Link to article:  https://theprint.in/india/whatsapp-group-tracker-mole-how-transport-mafia-planned-attack-on-ambala-ias-officer/508719/
Articles added:  17
Link is already present in the dataframe
Link to article:  https://theprint.in/plugged-in/newspapers-all-cheer-for-ins-arihant-and-indias-nuclear-triad/145907/
Articles added:  17
Article is not about current topic
Link to article:  https://theprint.in/india/minor-boys-from-ne-sexually-abused-in-ups-muzaffarnagar-ashram-godman-disciple-arrested/459187/
Articles added:  17
Article is not about current topic
Link to article:  https://theprint.in/india/congress-releases-list-of-86-candidates-for-kerala-assembly-polls/621786/
Articles added:  17
Link is already present in the dataframe
Link to article:  https://theprint.in/india/supreme-court-directs-kerala-govt-to-draw-up-law-on-administration-of-sabarimala-temple/323744/
Articles added:  17
Article is not about current topic
Link

Articles added:  19
Article is not about current topic
Link to article:  https://theprint.in/last-laughs/section-377-at-supreme-courts-doorstep-and-mumbais-latest-local-transport/82225/
Articles added:  19
Article is not about current topic
Link to article:  https://theprint.in/plugged-in/terrorism-has-no-religion-says-ravish-kumar-india-todays-questions-on-dsp-davinder-singh/349739/
Articles added:  19
Link is already present in the dataframe
Link to article:  https://theprint.in/theprint-essential/what-the-sabarimala-review-petition-supreme-court-will-rule-on-today-is-all-about/320668/
Articles added:  19
Article is not about current topic
Link to article:  https://theprint.in/judiciary/sc-stays-gujarat-hc-order-granting-two-week-furlough-to-asaram-bapus-son-narayan-sai/713710/
Articles added:  19
Link is already present in the dataframe
Link to article:  https://theprint.in/opinion/forgive-me-liberal-friends-but-i-cant-completely-overlook-faith-of-sabarimala-devotees/174990/
Article

Articles added:  24
Link is already present in the dataframe
Link to article:  https://theprint.in/judiciary/sc-9-judge-bench-on-sabarimala-to-now-hear-larger-issues-of-faith-vs-fundamental-rights/348671/
Articles added:  24
Link is already present in the dataframe
Link to article:  https://theprint.in/pre-truth/president-kovind-leaves-many-amused-how-a-wedding-invite-changed-rahul-gandhis-plans/92983/
Articles added:  24
Article is not about current topic
Link to article:  https://theprint.in/india/role-played-by-the-crucial-akhil-bharatiya-pratinidhi-sabha-of-rss-which-meets-this-week/620557/
Articles added:  24
Articles added:  25
Article is not about current topic
Link to article:  https://theprint.in/opinion/a-wish-list-your-honour-how-the-indian-judiciary-can-get-back-its-prestige-in-2019/171142/
Articles added:  25
Link is already present in the dataframe
Link to article:  https://theprint.in/india/faced-with-financial-crunch-sabarimala-1200-kerala-temples-plan-to-monetise-gold/

Articles added:  28
Article is not about current topic
Link to article:  https://theprint.in/india/rajkot-says-94-covid-deaths-in-3-months-but-crematorium-burial-ground-data-puts-it-at-1247/510172/
Articles added:  28
Article is not about current topic
Link to article:  https://theprint.in/opinion/mourn-idea-of-india-but-dont-forget-idea-of-people/488884/
Articles added:  28
Article is not about current topic
Link to article:  https://theprint.in/politics/between-the-lines-of-a-speech-by-chief-of-apolitical-rss-lies-an-overtly-political-pitch/136637/
Articles added:  28
Link is already present in the dataframe
Link to article:  https://theprint.in/india/39-covid-cases-in-sabarimala-since-pilgrim-season-began-last-week-temple-to-ramp-up-testing/553629/
Articles added:  28
Articles added:  29
Article is not about current topic
Link to article:  https://theprint.in/opinion/can-india-be-a-hindu-rashtra-and-hindus-a-minority-at-the-same-time/176361/
Articles added:  29
Article is not about 

Articles added:  34
Article is not about current topic
Link to article:  https://theprint.in/plugged-in/virat-kohli-end-of-the-world-moment-in-express-aatish-taseer-cries-foul-in-india-today/320850/
Articles added:  34
Articles added:  35
Link is already present in the dataframe
Link to article:  https://theprint.in/opinion/by-siding-with-sabarimala-devotees-shashi-tharoor-has-failed-the-liberal-in-him/176907/
Articles added:  35
Link is already present in the dataframe
Link to article:  https://theprint.in/opinion/telescope/ranveer-singh-deepika-padukone-wedding-has-finally-edged-out-breathless-rafale-debates/149550/
Articles added:  35
Link is already present in the dataframe
Link to article:  https://theprint.in/politics/in-an-u-turn-rahul-gandhi-now-says-he-cannot-take-an-open-and-shut-position-on-sabarimala/177555/
Articles added:  35
Articles added:  36
Link is already present in the dataframe
Link to article:  https://theprint.in/india/governance/two-women-begin-trekking-up-saba

Articles added:  39
Article is not about current topic
Link to article:  https://theprint.in/opinion/udf-had-a-chance-in-kerala-then-congress-played-a-dangerous-communal-game/627244/
Articles added:  39
Article is not about current topic
Link to article:  https://theprint.in/india/bjp-mp-subramanian-swamy-now-wants-to-reclaim-kashi-vishwanath-temple/501724/
Articles added:  39
Article is not about current topic
Link to article:  https://theprint.in/features/10-apps-to-take-you-from-casual-sex-to-marriage-vows/183478/
Articles added:  39
Link is already present in the dataframe
Link to article:  https://theprint.in/india/as-sabarimala-opens-kerala-cops-send-back-10-women-of-menstruating-age/322203/
Articles added:  39
Link is already present in the dataframe
Link to article:  https://theprint.in/last-laughs/rahuls-rafale-roar-congress-double-standards-on-sabarimala-and-it-raids-in-sandalwood/173061/
Articles added:  39
Article is not about current topic
Link to article:  https://theprin

Articles added:  43
Link is already present in the dataframe
Link to article:  https://theprint.in/india/governance/another-sabarimala-like-crisis-is-brewing-in-kerala/163897/
Articles added:  43
Article is not about current topic
Link to article:  https://theprint.in/politics/india-belongs-to-hindus-says-rss-chief-mohan-bhagwat-bats-for-bjp-in-temple-row/186637/
Articles added:  43
Article is not about current topic
Link to article:  https://theprint.in/plugged-in/give-peace-and-sidhu-a-chance-says-imran-khan/102735/
Articles added:  43
Articles added:  44
Article is not about current topic
Link to article:  https://theprint.in/judiciary/firm-meticulous-justice-indu-malhotra-retires-after-short-sc-stint-of-less-than-3-years/621376/
Articles added:  44
Articles added:  45
Link is already present in the dataframe
Link to article:  https://theprint.in/national-interest/bjp-could-be-shrinking-politically-but-is-winning-big-time-ideologically/328397/
Articles added:  45
Link is already pre

In [ ]:
#Saving the dataframe to disk
theprint_df.to_pickle("theprint_dataframe", compression="zip")
theprint_df.shape

(2569, 6)

#### 6. CAA

In [ ]:
#Setting the topic
topic = 'caa'

#Unpickling the links
links_list = links_unpickle(topic)

Number of links on the caa topic is 1826.


In [ ]:
#Removing duplicate links
links_list = list(set(links_list))

#Dropping links that have the term 'video' in them
links_list = [l for l in links_list if 'video' not in l]
print("Number of links after cleaning: ", len(links_list))

Number of links after cleaning:  1789


Adding these articles to the dataframe.

In [ ]:
#Collecting the links that were not added to the dataframe
topic_other_list_6, exceptions_links_list_6 = link_to_article(links_list, topic)

Article is not about current topic
Link to article:  https://theprint.in/india/zuckerberg-uses-kapil-mishras-delhi-riots-threat-to-outline-facebooks-hate-speech-policy/435845/
Articles added:  0
Link is already present in the dataframe
Link to article:  https://theprint.in/politics/shashi-tharoors-article-kicks-up-a-new-storm-on-sabarimala-splits-opinion/148163/
Articles added:  0
Articles added:  1
Article is not about current topic
Link to article:  https://theprint.in/opinion/india-blames-external-forces-for-export-mess-but-problem-is-internal-budget-2020-can-help/354860/
Articles added:  1
Article is not about current topic
Link to article:  https://theprint.in/opinion/akhilesh-yadav-is-counting-his-chickens-before-they-hatch/366788/
Articles added:  1
Articles added:  2
Link is already present in the dataframe
Link to article:  https://theprint.in/india/postpaid-mobile-services-in-kashmir-likely-to-resume-from-monday/304896/
Articles added:  2
Article is not about current topic
Li

Articles added:  16
Article is not about current topic
Link to article:  https://theprint.in/opinion/nhrc-chief-hl-dattu-will-quit-office-this-year-with-him-the-rights-body-must-wind-up-too/429996/
Articles added:  16
Article is not about current topic
Link to article:  https://theprint.in/plugged-in/mirror-now-news-18-focus-on-bengaluru-riots-abps-honey-trap-twist-in-sushant-case/480630/
Articles added:  16
Article is not about current topic
Link to article:  https://theprint.in/opinion/indian-media-waging-holy-war-against-muslims-hyenas/400407/
Articles added:  16
Link is already present in the dataframe
Link to article:  https://theprint.in/theprint-essential/the-religious-significance-of-the-18-steps-at-sabarimala-temple/173206/
Articles added:  16
Article is not about current topic
Link to article:  https://theprint.in/india/up-govt-stubborn-like-a-child-can-frame-me-in-another-case-says-kafeel-khan-after-release/493925/
Articles added:  16
Article is not about current topic
Link 

Articles added:  39
Articles added:  40
Articles added:  41
Link is already present in the dataframe
Link to article:  https://theprint.in/india/sc-seeks-response-from-centre-jammu-kashmir-on-plea-seeking-to-produce-farooq-abdullah/292026/
Articles added:  41
Articles added:  42
Link is already present in the dataframe
Link to article:  https://theprint.in/opinion/section-377-ruling-supreme-court-has-evolved-but-politicians-need-to-play-catch-up/112694/
Articles added:  42
Article is not about current topic
Link to article:  https://theprint.in/opinion/why-its-so-easy-for-modi-to-appropriate-tagore-or-anyone/594094/
Articles added:  42
Articles added:  43
Link is already present in the dataframe
Link to article:  https://theprint.in/india/only-40-gst-returns-filed-in-jk-due-to-internet-blockade-big-fall-in-revenue-collection/323492/
Articles added:  43
Articles added:  44
Articles added:  45
Article is not about current topic
Link to article:  https://theprint.in/opinion/lets-talk-abou

Articles added:  63
Article is not about current topic
Link to article:  https://theprint.in/india/yogendra-yadav-ram-guha-urge-nationalisation-of-property-to-fight-covid-step-back-after-row/427898/
Articles added:  63
Link is already present in the dataframe
Link to article:  https://theprint.in/statedraft/next-door-to-sabarimala-temple-district-a-menstruating-goddess-is-worshipped/173322/
Articles added:  63
Article is not about current topic
Link to article:  https://theprint.in/plugged-in/chidambaram-shivkumar-silly-old-fools-arnab-media-see-red-in-growth-blues/286323/
Articles added:  63
Link is already present in the dataframe
Link to article:  https://theprint.in/features/from-rbi-cbi-to-metoo-rafale-here-are-the-top-10-newsmakers-of-2018/170191/
Articles added:  63
Article is not about current topic
Link to article:  https://theprint.in/plugged-in/primetime/corona-gyaan-across-channels-tharoor-sibal-as-head-shoulders-on-times-now/380235/
Articles added:  63
Article is not about

Articles added:  80
Article is not about current topic
Link to article:  https://theprint.in/best-of-theprint-icymi/how-a-woman-who-couldnt-go-to-iit-because-she-got-married-became-isros-top-engineer/113965/
Articles added:  80
Article is not about current topic
Link to article:  https://theprint.in/opinion/global-print/from-modi-to-imran-khan-trumps-visit-to-delhi-holds-a-victory-for-all-sides/352380/
Articles added:  80
Link is already present in the dataframe
Link to article:  https://theprint.in/india/supreme-court-asks-centre-to-restore-normalcy-in-kashmir/292013/
Articles added:  80
Articles added:  81
Link is already present in the dataframe
Link to article:  https://theprint.in/india/governance/govt-to-procure-apples-directly-from-jammu-kashmir-farmers/289442/
Articles added:  81
Articles added:  82
Articles added:  83
Articles added:  84
Article is not about current topic
Link to article:  https://theprint.in/opinion/indias-young-are-no-more-charmed-by-modi-economy-and-ideolog

...

Articles added:  544
Articles added:  545
Articles added:  546
Article is not about current topic
Link to article:  https://theprint.in/india/governance/rajnath-raises-issue-of-absentee-mps-in-parliament-as-govt-prepares-to-table-citizenship-bill/329544/
Articles added:  546
Article is not about current topic
Link to article:  https://theprint.in/plugged-in/slump-in-markets-crude-oil-prices-coronavirus-political-turmoil-in-mp-rule-front-pages/378417/
Articles added:  546
Article is not about current topic
Link to article:  https://theprint.in/india/up-cop-accuses-5-ips-officers-of-corruption-ganging-up-with-extortionists-in-letter-to-dgp/344316/
Articles added:  546
Articles added:  547
Articles added:  548
Article is not about current topic
Link to article:  https://theprint.in/india/indian-tv-medias-chandrayaan-2-coverage-modi-pakistan-deepak-chaurasias-tinfoil-suit/288460/
Articles added:  548
Article is not about current topic
Link to article:  https://theprint.in/politics/priyanka

Articles added:  571
Articles added:  572
Article is not about current topic
Link to article:  https://theprint.in/iwitness/observing-big-changes-digging-in-to-the-best-cuisine-humming-kishore-on-assam-poll-trail/630493/
Articles added:  572
Article is not about current topic
Link to article:  https://theprint.in/india/man-arrested-for-hatching-conspiracy-to-incite-communal-riots-in-northeast-delhi/378006/
Articles added:  572
Article is not about current topic
Link to article:  https://theprint.in/opinion/family-elder-modi-not-aggressive-amit-shah-govts-face-in-covid-19-fight/391611/
Articles added:  572
Articles added:  573
Articles added:  574
Article is not about current topic
Link to article:  https://theprint.in/opinion/reclaiming-indian-pluralism-will-need-annihilation-of-congress/485212/
Articles added:  574
Articles added:  575
Article is not about current topic
Link to article:  https://theprint.in/world/all-licenses-issued-to-commercial-pilots-genuine-says-pakistan-aviation-

Articles added:  605
Article is not about current topic
Link to article:  https://theprint.in/opinion/pov/uncle-please-sit-jaggi-vasudev-needs-to-shut-up-on-things-he-has-no-clue-about/367400/
Articles added:  605
Article is not about current topic
Link to article:  https://theprint.in/politics/jobs-shaheen-bagh-focus-of-rahul-and-priyanka-gandhis-first-joint-rally-outside-up/359742/
Articles added:  605
Articles added:  606
Articles added:  607
Article is not about current topic
Link to article:  https://theprint.in/opinion/bjp-and-congress-use-tipu-sultan-for-politics-but-are-shockingly-ignorant-of-his-history/147767/
Articles added:  607
Article is not about current topic
Link to article:  https://theprint.in/diplomacy/indias-diversity-should-be-kept-as-it-is-new-french-ambassador-emmanuel-lenain/336273/
Articles added:  607
Articles added:  608
Article is not about current topic
Link to article:  https://theprint.in/opinion/plague-tested-gandhi-and-patel-leadership-style-just-like-

Articles added:  618
Article is not about current topic
Link to article:  https://theprint.in/opinion/move-over-angry-young-man-2018-was-ruled-by-angry-young-women-in-bollywood/168598/
Articles added:  618
Article is not about current topic
Link to article:  https://theprint.in/politics/article-370-modi-take-backseat-in-haryana-elections-as-jobs-caste-dominate-poll-scene/307884/
Articles added:  618
Article is not about current topic
Link to article:  https://theprint.in/opinion/not-modi-not-yogi-its-ram-versus-rest-for-bjp-in-up-2022/479085/
Articles added:  618
Article is not about current topic
Link to article:  https://theprint.in/plugged-in/republic-on-sushant-murder-times-now-super-stunner-letter-tribute-to-rahat-indori/479923/
Articles added:  618
Articles added:  619
Link is already present in the dataframe
Link to article:  https://theprint.in/plugged-in/todays-news-imrans-interviews-on-nuclear-war-kashmir-godavari-deaths-yogi-on-nrc/292091/
Articles added:  619
Article is not

Articles added:  643
Articles added:  644
Articles added:  645
Article is not about current topic
Link to article:  https://theprint.in/india/jnu-students-have-time-and-again-upheld-what-it-means-to-be-a-national-university/350320/
Articles added:  645
Link is already present in the dataframe
Link to article:  https://theprint.in/india/modi-govt-has-sent-rs-800-cr-to-jk-panchayats-but-no-projects-planned-yet-to-spend-it/286945/
Articles added:  645
Article is not about current topic
Link to article:  https://theprint.in/politics/tmc-appeals-to-left-congress-to-support-mamata-banerjee-against-divisive-bjp/584773/
Articles added:  645
Link is already present in the dataframe
Link to article:  https://theprint.in/opinion/section-377-exposes-bjp-and-congress-doublespeak-on-homosexuality/81783/
Articles added:  645
Article is not about current topic
Link to article:  https://theprint.in/politics/bjp-will-win-26-of-30-bengal-seats-that-voted-in-1st-phase-says-amit-shah/630139/
Articles added

In [ ]:
#Saving the dataframe to disk
theprint_df.to_pickle("theprint_dataframe", compression="zip")
theprint_df.shape

(3216, 6)

In [ ]:
#To load the pickled dataframe
theprint_df = pd.read_pickle("theprint_dataframe", compression="zip")
theprint_df.shape

(3216, 6)

Let's check if there are any rows where the article wasn't picked up by the code.

In [ ]:
theprint_df[(theprint_df['Title'] == 'no match') | (theprint_df['Date'] == 'no match') 
              | (theprint_df['Authors'] == 'no match') | (theprint_df['Article'] == 'no match')]

,Title,Link,Date,Authors,Topic,Article
690,Punjab CM Amarinder Singh urges farmers to vac...,https://theprint.in/india/punjab-cm-amarinder-...,no match,no match,farm laws,Chandigarh : Punjab Chief Minister Amarinder S...


In [ ]:
#Dropping this row
theprint_df.drop([690], inplace=True)

Let's select a few articles at random to check.

In [ ]:
sample_df = theprint_df.sample(n=10)
sample_df

,Title,Link,Date,Authors,Topic,Article
1111,"Congress outs new page of MoD Rafale note, hin...",https://theprint.in/defence/congress-outs-new-...,"9 February, 2019 1:29 pm",Snehesh Alex Philip,rafale,A Union Defence Ministry note from 2015 has tr...
1489,"If India is in crisis, it is because good guys...",https://theprint.in/opinion/if-india-is-in-cri...,"27 August, 2019 8:59 am",Nitin Pai,article 370,"To arrest India’s decline, it is important tha..."
2190,Kashmiris are dodging internet shutdown to wat...,https://theprint.in/india/kashmiris-are-dodgin...,"12 December, 2019 11:36 am",Azaan Javaid,article 370,Dirilis Ertugrul is a historical fiction serie...
1573,"J&amp;K leaders Sajad Lone, Waheed Parra relea...",https://theprint.in/india/jk-leaders-sajad-lon...,"5 February, 2020 2:51 pm",PTI,article 370,People's Conference leader Sajad Lone and PDP'...
39,Farmer unions call for Bharat bandh on 26 March,https://theprint.in/india/farmer-unions-call-f...,"10 March, 2021 9:23 pm",PTI,farm laws,The day will mark four months of the farmers’ ...
270,Draconian Essential Commodities Act needs to g...,https://theprint.in/opinion/draconian-essentia...,"25 January, 2019 10:41 am",Ila Patnaik,farm laws,The fear of the Essential Commodities Act may ...
1020,Rafale shows HAL desperately needs competition,https://theprint.in/50-word-edit/rafale-shows-...,"3 October, 2018 6:37 pm",ThePrint Team,rafale,The 50-word Edit — ThePrint view on the most...
587,"65-yr-old ‘attempts suicide’ at Singhu border,...",https://theprint.in/india/65-yr-old-attempts-s...,"21 December, 2020 9:14 pm",Sravasti Dasgupta,farm laws,Doctors at PGIMS Rohtak say condition of Niran...
1139,Postpaid mobile services in Kashmir likely to ...,https://theprint.in/india/postpaid-mobile-serv...,"12 October, 2019 12:58 pm",PTI,article 370,J&amp;K administration officials say postpaid ...
2196,"Tweets, tantrums, half-truths – Pakistan minis...",https://theprint.in/opinion/tweets-tantrums-ha...,"3 September, 2019 11:00 am",Akshobh Giridharadas,article 370,Pakistan foreign minister Shah Mahmood Qureshi...


In [ ]:
print(theprint_df['Link'][2196])
print(theprint_df['Article'][2196])

https://theprint.in/opinion/tweets-tantrums-half-truths-pakistan-minister-shah-mahmood-qureshis-strategy-for-kashmir/285718/
Pakistan foreign minister Shah Mahmood Qureshi has long been seen as a Kashmir hawk, but the latest crisis has exposed his inadequacies only too well.P akistan’s foreign minister Shah Mahmood Qureshi has been at the centre of several crises that have altered India-Pakistan relationship in the recent years. He was the foreign minister  during  the Mumbai 26/11 terror attacks in 2008. His second stint, this time in the Imran Khan government, has been rocked by the Pulwama attack, the Balakot airstrikes and now the Modi government’s decision to abrogate Article 370 for Jammu and Kashmir. But these moments of tensions appear to have done nothing to sharpen his diplomacy in the times of crisis. Shah Mahmood Qureshi seems to have no concrete policy to deal with the situation arising out of revocation of Article 370. He appears to be more preoccupied with bombastic and 

Next, let's check if the articles on each topic contain the relevant keywords.

In [ ]:
#Initializing an empty list to collect the indices of the articles that are to be dropped
articles_to_drop = []

In [ ]:
#Checking the articles on farm laws
theprint_df[(theprint_df['Topic'] == 'farm laws') & ~theprint_df['Article'].str.contains('farm|agri', case=False) & ~theprint_df['Title'].str.contains('farm|agri', case=False)]

,Title,Link,Date,Authors,Topic,Article
59,Deepak Chaurasia says US should learn from Ind...,https://theprint.in/plugged-in/primetime/deepa...,"8 January, 2021 9:21 am",Angana Chakrabarti,farm laws,A quick take on what prime time TV news talked...
351,Delhi’s ‘customary welcome’ &amp; govt’s ‘tric...,https://theprint.in/last-laughs/delhis-customa...,"27 November, 2020 6:04 pm",Tenzin Zompa,farm laws,"The best cartoons of the day, chosen by the ed..."
483,"90% of milk in India comprises A2 protein, say...",https://theprint.in/best-of-theprint-icymi/90-...,"30 January, 2021 3:39 pm",ThePrint Team,farm laws,"A selection of the best news reports, analysis..."
726,Supreme Court upholds power of Arvind Kejriwal...,https://theprint.in/plugged-in/supreme-court-u...,"5 July, 2018 8:18 am",Nandita Singh,farm laws,Business Class Reliance Jio gets ready for the...


In [ ]:
print(theprint_df['Link'][726])
print(theprint_df['Article'][726])

https://theprint.in/plugged-in/supreme-court-upholds-power-of-arvind-kejriwal-govt-and-whatsapp-is-horrified/79142/
Business Class Reliance Jio gets ready for the next round of disruption in the telecom sector. The company informed the government that it will be launching voice over WiFi services very soon, thus enabling calls in areas with poor mobile signals,  reports   The Economic Times . Remember the fanfare with which Tata Nano was launched in 2008? India’s ‘magic vehicle’ is nearing its end with just one unit produced last month,  reports   The Economic Times . The entry-level car sold just three units in June in the domestic market. Point of View The Supreme Court Wednesday restored the authority of the elected government in Delhi in administrative matters. The Indian Express, in its  editorial,  writes, “SC upholds a basic democratic principle: Elected governments have the upper hand.” The Hindu added in its editorial, “The SC clarifies an elected government cannot be undermin

In [ ]:
articles_to_drop.append(726)

In [ ]:
len(articles_to_drop)

2

Let's also drop all articles that are on primetime and last-laughs.

In [ ]:
primetime_indices = theprint_df.index[theprint_df['Link'].str.contains('primetime')]
len(primetime_indices)

26

In [ ]:
laughs_indices = theprint_df.index[theprint_df['Link'].str.contains('last-laugh')]
len(laughs_indices)

104

In [ ]:
#Dropping these articles
print("Number of articles before dropping: ", theprint_df.shape[0])

Number of articles before dropping:  3187


In [ ]:
theprint_df.drop(articles_to_drop, inplace=True)
theprint_df.drop(primetime_indices, inplace=True)

In [ ]:
theprint_df.drop(laughs_indices, inplace=True)
print("Number of articles after dropping: ", theprint_df.shape[0])

Number of articles after dropping:  3083


In [ ]:
#Checking the articles on rafale
theprint_df[(theprint_df['Topic'] == 'rafale') & ~theprint_df['Article'].str.contains('rafale|dassault', case=False) & ~theprint_df['Title'].str.contains('rafale|dassault', case=False)]

,Title,Link,Date,Authors,Topic,Article
787,"To keep IAF flying, India will pull aircraft f...",https://theprint.in/india/governance/to-keep-i...,"18 October, 2018 9:37 am",Sujan Dutta,rafale,India is the only air force in the world still...
882,"Clean energy, tech in focus as France looks be...",https://theprint.in/defence/clean-energy-tech-...,"29 March, 2021 11:03 am",Snehesh Alex Philip,rafale,France is a significant source of FDI in India...
981,Rahul Gandhi must decide if Congress is a Euro...,https://theprint.in/opinion/rahul-gandhi-must-...,"23 May, 2019 3:44 pm",Abhijit Iyer-Mitra,rafale,Safe to say national security plank will alway...


In [ ]:
#Initializing an empty list to collect the indices of the articles that are to be dropped
articles_to_drop = []

In [ ]:
print(theprint_df['Link'][981])
print(theprint_df['Article'][981])

https://theprint.in/opinion/rahul-gandhi-must-decide-if-congress-is-a-european-ngo-or-a-party-in-nationalist-india/239546/
Safe to say national security plank will always favour the BJP, unless Rahul Gandhi replaces his national security advisers.I ndian elections and national security have a seemingly unpredictable relationship. The defeat in the 1962 war with China and the victory in the 1971 war happened immediately after elections earlier in the year and can’t be used as gauges. However, the Sri Lanka fiasco and Bofors took a heavy toll on Rajiv Gandhi,  wiping out  his government in the 1989 Lok Sabha elections. The victory in the Kargil War propelled Atal Bihari Vajpayee to a  victory  in 1999. The 26/11 Mumbai attacks were undeniably a massive intelligence and security failure, and yet  in state elections  a few days later, the Congress trounced the BJP. Confused? So am I. While overarching trends are difficult to make out, there are some allegories that bear repeating. Subraman

In [ ]:
articles_to_drop.append(981)

In [ ]:
len(articles_to_drop)

3

In [ ]:
#Checking the articles on article 370
theprint_df[(theprint_df['Topic'] == 'article 370') & ~theprint_df['Article'].str.contains('370|kashmir|j&k', case=False) & ~theprint_df['Title'].str.contains('370|kashmir|j&k', case=False)]

,Title,Link,Date,Authors,Topic,Article
1422,"Hair rebellion — from Kautilya, Omar Abdullah ...",https://theprint.in/opinion/pov/hair-rebellion...,"26 April, 2020 1:34 pm",Shubhangi Misra,article 370,"To deal with boredom, anxiety and restlessness..."
1599,Modi govt needs to offer clarity on why it red...,https://theprint.in/50-word-edit/modi-govt-nee...,"5 August, 2019 8:12 pm",ThePrint Team,article 370,"ThePrint view on the most important issues, in..."
1838,Ram Mandir can’t wait for Covid vaccine. Modi ...,https://theprint.in/opinion/ram-mandir-cant-wa...,"28 July, 2020 8:52 am",Ruhi Tewari,article 370,"For BJP, Ayodhya issue is the most significant..."
1892,NIA files chargesheet against six people for h...,https://theprint.in/india/nia-files-chargeshee...,"27 July, 2020 9:10 pm",PTI,article 370,The accused have been charged with relevant pr...
2206,"3 militants, 1 civilian killed as search opera...",https://theprint.in/india/3-militants-1-civili...,"17 September, 2020 12:10 pm",PTI,article 370,"Two CRPF personnel, including an officer, sust..."


In [ ]:
print(theprint_df['Link'][2206])
print(theprint_df['Article'][2206])

https://theprint.in/india/3-militants-1-civilian-killed-as-search-operation-turns-into-encounter-in-jks-batamaloo/504547/
Two CRPF personnel, including an officer, sustained injuries after militants opened fire during the search operation in Firdousabad locality. Three militants and a civilian woman were killed, while two CRPF personnel, including an officer, sustained injuries in a predawn gunbattle on Thursday in Batamaloo area of the city, officials said. Security forces launched a cordon and search operation in Firdousabad locality in Batamaloo area at around 2.30 am following information about the presence of militants there, the officials said. They said the search operation turned into an encounter after militants opened fire on security forces. One civilian identified as Kaunsar Riyaz was killed in the firing, while two CRPF personnel, including an officer, were injured, the officials said. The injured personnel were taken to hospital and the operation was still in progress, th

In [ ]:
articles_to_drop.append(2206)

In [ ]:
len(articles_to_drop)

7

In [ ]:
#Checking the articles on sabarimala
theprint_df[(theprint_df['Topic'] == 'sabarimala') & ~theprint_df['Article'].str.contains('sabarimala|ayyappa', case=False) & ~theprint_df['Title'].str.contains('sabarimala|ayyappa', case=False)]

,Title,Link,Date,Authors,Topic,Article
2462,"Adultery may not be cause of unhappy marriage,...",https://theprint.in/india/governance/adultery-...,"28 September, 2018 1:26 pm",Ritika Jain,sabarimala,The judgment delivered by Supreme Court was re...


In [ ]:
print(theprint_df['Link'][2462])
print(theprint_df['Article'][2462])

https://theprint.in/india/governance/adultery-may-not-be-cause-of-unhappy-marriage-can-be-result-quotes-from-scs-497-ruling/126170/
The judgment delivered by Supreme Court was replete with observations that vindicated rights patriarchal societies have denied women for decades.  As the Supreme Court struck down Section 497 of the IPC, a sexist law on adultery, Thursday, Chief Justice of India Dipak Misra invoked an evocative analogy to emphasise the equality of women. “…The essentiality of the rights of women gets the real requisite space in the living room of individual dignity rather than… in an annexe to the main building,” he wrote, penning the judgment for himself and Justice A.M. Khanwilkar. A five-judge Constitution bench was unanimous in striking down the colonial-era law, which absolved men of guilt if they courted a married woman with her husband’s permission. Another controversial provision kept women out of the purview of punishment, with only men held accountable. In their 

In [ ]:
articles_to_drop.append(2462)

In [ ]:
#Checking the articles on section 377
theprint_df[(theprint_df['Topic'] == 'section 377') & ~theprint_df['Article'].str.contains('377|queer|lgbt|gay', case=False) & ~theprint_df['Title'].str.contains('377|queer|lgbt|gay', case=False)]

,Title,Link,Date,Authors,Topic,Article
2532,"Thank you, Supreme Court",https://theprint.in/50-word-edit/thank-you-sup...,"6 September, 2018 1:14 pm",ThePrint Team,section 377,The 50-word Edit — ThePrint view on the most...


In [ ]:
print(theprint_df['Link'][2532])
print(theprint_df['Article'][2532])

https://theprint.in/50-word-edit/thank-you-supreme-court/112826/
The  50-word Edit  — ThePrint view on the most important issues, instantly. The Supreme Court has redeemed itself by decriminalising homosexuality. This is a historic win for civil rights and equality, especially because it addresses a longstanding grievance of a minority group. Politicians, however, should be filled with remorse for shirking their responsibility and allowing the judiciary to claim all credit.


In [ ]:
#Checking the articles on CAA
theprint_df[(theprint_df['Topic'] == 'CAA') & ~theprint_df['Article'].str.contains('caa|nrc|citizen|shaheen', case=False) & ~theprint_df['Title'].str.contains('caa|nrc|citizen|shaheen', case=False)]

,Title,Link,Date,Authors,Topic,Article


In [ ]:
#Dropping the identified articles
print("Number of articles before dropping: ", theprint_df.shape[0])
theprint_df.drop(articles_to_drop, inplace=True)
print("Number of articles after dropping: ", theprint_df.shape[0])

Number of articles before dropping:  3083
Number of articles after dropping:  3075


In [ ]:
#Saving the dataframe to disk
theprint_df.to_pickle("theprint_dataframe", compression="zip")
theprint_df.shape

(3075, 6)

In [ ]:
#To load the pickled dataframe
theprint_df = pd.read_pickle("theprint_dataframe", compression="zip")
theprint_df.shape

(3075, 6)

In [ ]:
print("Farm Laws", theprint_df[theprint_df['Topic']=='farm laws'].shape)
print("Rafale", theprint_df[theprint_df['Topic']=='rafale'].shape)
print("Article 370", theprint_df[theprint_df['Topic']=='article 370'].shape)
print("Sabarimala", theprint_df[theprint_df['Topic']=='sabarimala'].shape)
print("Section 377", theprint_df[theprint_df['Topic']=='section 377'].shape)
print("CAA", theprint_df[theprint_df['Topic']=='caa'].shape)

Farm Laws (701, 6)
Rafale (354, 6)
Article 370 (1253, 6)
Sabarimala (100, 6)
Section 377 (43, 6)
CAA (624, 6)


In [ ]:
theprint_df['Authors'].value_counts()[:8]

PTI                    828
Azaan Javaid           252
ThePrint Team          159
Snehesh Alex Philip     87
Nayanima Basu           83
Ananya Bhardwaj         70
Fatima Khan             62
ANI                     56
Name: Authors, dtype: int64

In [ ]:
theprint_df[theprint_df['Authors'].str.contains('Press|News|PTI|ANI|UTI|IANS', case=False)].shape

(1019, 6)